In [1]:
class RadixTree:
    """
    Uma árvore de prefixos compacta.
    """

    def __init__(self) -> None:
        """
        Cria uma árvore de prefixos compacta vazia.
        >>> RadixTree()
        """
        # O nó raiz da árvore
        self.root = RadixNode()
        # O número de palavras na árvore
        self.size = 0
        # Próximo índice
        self.next_index = 0

    def insert(self, word: str) -> None:
        """
        Insere uma palavra na árvore.

        Args:
            word (str): palavra a ser inserida.

        >>> RadixTree().insert("word")
        """
        index = self.next_index
        self.root.insert(word, index)
        self.next_index += 1
        self.size += 1

    def insert_many(self, words: list[str]) -> None:
        """
        Insere várias palavras na árvore.

        Args:
            words (list[str]): lista de palavras a serem inseridas.

        >>> RadixTree().insert_many(["word1", "word2"])
        """
        for word in words:
            self.insert(word)

    def find(self, word: str) -> tuple[bool, int]:
        """
        Verifica se uma palavra está na árvore e retorna seu índice.

        Args:
            word (str): palavra a ser verificada.

        Returns:
            bool: True se a palavra estiver na árvore,
                  False caso contrário.
            int: Índice da palavra,
                 -1 caso a palavra não encontrada.

        >>> RadixTree().insert("word")
        >>> RadixTree().find("word")
        True, 0
        """
        return self.root.find(word)

    def index(self, word: str) -> int:
        """
        Retorna o índice de uma palavra na árvore.

        Args:
            word (str): palavra a ser verificada.

        Returns:
            int: índice da palavra na árvore.

        >>> RadixTree().insert("word")
        >>> RadixTree().index("word")
        0
        """
        return self.find(word)[1]

    def __getitem__(self, word: str) -> int:
        """
        Retorna o índice de uma palavra na árvore.

        Args:
            word (str): palavra a ser verificada.

        Returns:
            int: índice da palavra na árvore.

        >>> RadixTree().insert("word")
        >>> RadixTree()["word"]
        0
        """
        return self.index(word)

    def delete(self, word: str) -> bool:
        """
        Deleta uma palavra da árvore.

        Args:
            word (str): palavra a ser deletada.

        Returns:
            bool: True caso a palavra esteja na árvore e for deletada,
                  False caso a palavra não esteja na árvore.

        >>> RadixTree().insert("word")
        >>> RadixTree().delete("word")
        True
        """
        if self.root.delete(word):
            self.size -= 1
            return True
        return False

    def print_tree(self) -> None:
        """
        Imprime a árvore.

        >>> RadixTree().insert_many(["word1", "word2"])
        >>> RadixTree().print_tree()
        --- word
        ------ 1:   (0)
        ------ 2:   (1)
        """
        self.root.print_tree()

    def all_words(self) -> dict:
        """
        Retorna todas as palavras da árvore numa lista.

        Returns:
            list[(str, int)]: lista de palavras e índices.

        >>> RadixTree().insert_many(["word1", "word2"])
        >>> RadixTree().all_words()
        [('word1', 0), ('word2', 1)]
        """
        return self.root.all_words()

    def __repr__(self) -> str:
        """
        >>> RadixTree()
        'RadixTree(size=0)'
        """
        return f"RadixTree(size={self.size})"

    def __str__(self) -> str:
        """
        >>> RadixTree().insert_many(["word1", "word2"])
        >>> print(RadixTree())
        {'word1': 0, 'word2': 1}
        """
        return self.all_words().__str__()

    def __contains__(self, word: str) -> bool:
        """
        >>> RadixTree().insert("word")
        >>> "word" in RadixTree()
        True
        """
        return self.find(word)[0]

    def __iter__(self):
        """
        >>> RadixTree().insert_many(["word1", "word2"])
        >>> [(word, i) for (word, i) in RadixTree()]
        [('word1', 0), ('word2', 1)]
        """
        return iter(self.all_words())

    def __add__(self, word: str) -> None:
        """
        >>> RadixTree() + "word"
        >>> "word" in RadixTree()
        True
        """
        self.insert(word)

    def __iadd__(self, word: str):
        """
        >>> RadixTree() += "word"
        >>> "word" in RadixTree()
        True
        """
        self.insert(word)
        return self

    def __sub__(self, word: str) -> bool:
        """
        >>> RadixTree().insert("word")
        >>> RadixTree() - "word"
        >>> "word" not in RadixTree()
        True
        """
        return self.delete(word)

    def __isub__(self, word: str):
        """
        >>> RadixTree().insert("word")
        >>> RadixTree() -= "word"
        >>> "word" not in RadixTree()
        True
        """
        self.delete(word)
        return self

    def __len__(self) -> int:
        """
        >>> RadixTree().insert_many(["word1", "word2"])
        >>> len(RadixTree())
        2
        """
        return self.size

    def sort(self, by="index") -> list[(str, int)]:
        """
        >>> RadixTree().insert_many(["word1", "word0"])
        >>> RadixTree().sort(by="index")
        [('word1', 0), ('word0', 1)]
        >>> RadixTree().sort(by="word")
        [('word0', 1), ('word1', 0)]
        """
        words = self.all_words()
        words = [(word, i) for word, i in words.items()]
        if by == "index":
            words.sort(key=lambda x: x[1])
        elif by == "word":
            words.sort(key=lambda x: x[0])
        return words


class RadixNode:
    """
    Um nó da árvore de prefixos compacta.
    """

    def __init__(
        self, prefix: str = "", is_leaf: bool = False, index: int = None
    ) -> None:
        """
        Cria um nó da árvore de prefixos compacta.

        Args:
            prefix (str): prefixo do nó.
            is_leaf (bool): se o nó é uma folha.
            index (int): índice da palavra adicionada

        >>> RadixNode()
        >>> RadixNode("prefix")
        """
        # Mapeamento a partir do primeiro caractere do prefixo do nó
        self.nodes: dict[str, RadixNode] = {}
        # Um nó será uma folha se a árvore conter sua palavra
        self.is_leaf = is_leaf
        self.prefix = prefix
        self.index = index

    def _match(self, word: str) -> tuple[str, str, str]:
        """
        Calcula a substring comum do prefixo do nó e uma palavra.

        Args:
            word (str): palavra para comparar.

        Returns:
            (str, str, str): substring comum, prefixo restante, palavra restante.

        >>> RadixNode("myprefix")._match("mystring")
        ('my', 'prefix', 'string')
        """
        x = 0
        for q, w in zip(self.prefix, word):
            if q != w:
                break

            x += 1

        return self.prefix[:x], self.prefix[x:], word[x:]

    def insert(self, word: str, index: int) -> None:
        # Caso 1: Se a palavra for o prefixo do nó
        # Solução: Definimos o nó atual como folha
        if self.prefix == word and not self.is_leaf:
            self.is_leaf = True

        # Caso 2: O nó não tem arestas que tenham um prefixo para a palavra
        # Solução: Criamos uma aresta do nó atual para um novo
        # contendo a palavra
        elif word[0] not in self.nodes:
            self.nodes[word[0]] = RadixNode(word, True, index)

        else:
            new_node = self.nodes[word[0]]
            substring, rest_prefix, rest_word = new_node._match(word)

            # Caso 3: O prefixo do nó é igual ao correspondente
            # Solução: Inserimos a palavra restante no próximo nó
            if rest_prefix == "":
                self.nodes[substring[0]].insert(rest_word, index)

            # Caso 4: A palavra é maior que a correspondência
            # Solução: Crie um nó entre os dois nós, altere
            # prefixos e adicione o novo nó para a palavra restante
            else:
                new_node.prefix = rest_prefix

                aux_node = self.nodes[substring[0]]
                self.nodes[substring[0]] = RadixNode(substring, False, index)
                self.nodes[substring[0]].nodes[rest_prefix[0]] = aux_node

                if rest_word == "":
                    self.nodes[substring[0]].is_leaf = True
                else:
                    self.nodes[substring[0]].insert(rest_word, index)

    def find(self, word: str) -> tuple[bool, int]:
        new_node = self.nodes.get(word[0], None)
        if not new_node:
            return (False, -1)
        else:
            substring, rest_prefix, rest_word = new_node._match(word)
            # Se houver prefixo restante, a palavra não pode estar na árvore
            if rest_prefix != "":
                return (False, -1)
            # Isso se aplica quando a palavra e o prefixo são iguais
            elif rest_word == "":
                return (True, new_node.index) if new_node.is_leaf else (False, -1)
            # Temos palavras restantes, então verificamos o próximo nó
            else:
                return new_node.find(rest_word)

    def delete(self, word: str) -> bool:
        new_node = self.nodes.get(word[0], None)
        if not new_node:
            return False
        else:
            substring, rest_prefix, rest_word = new_node._match(word)
            # Se houver prefixo restante, a palavra não pode estar na árvore
            if rest_prefix != "":
                return False, -1
            # Temos palavras restantes, então verificamos o próximo nó
            elif rest_word != "":
                return new_node.delete(rest_word)
            # Se não for uma folha, não precisamos excluir
            elif not new_node.is_leaf:
                return False
            else:
                # Excluímos os nós se nenhuma aresta sair deles
                if len(new_node.nodes) == 0:
                    del self.nodes[word[0]]
                    # Nós mesclamos o nó atual com seu único filho
                    if len(self.nodes) == 1 and not self.is_leaf:
                        merge_node = next(iter(self.nodes.values()))
                        self.is_leaf = merge_node.is_leaf
                        self.prefix += merge_node.prefix
                        self.index = merge_node.index
                        self.nodes = merge_node.nodes
                # Se houver mais de uma aresta, nós apenas a marcamos como não-folha
                elif len(new_node.nodes) > 1:
                    new_node.is_leaf = False
                # Se houver 1 aresta, nós a mesclamos com seu filho
                else:
                    merge_node = next(iter(new_node.nodes.values()))
                    new_node.is_leaf = merge_node.is_leaf
                    new_node.prefix += merge_node.prefix
                    new_node.index = merge_node.index
                    new_node.nodes = merge_node.nodes
                return True

    def print_tree(self, height: int = 0) -> None:
        if self.prefix != "":
            print(
                "---" * height,
                (
                    f"{self.prefix}:    ({self.index})"
                    if self.is_leaf
                    else f"{self.prefix}"
                ),
            )

        for value in self.nodes.values():
            value.print_tree(height + 1)

    def all_words(self) -> dict[int, str]:
        words = {}
        if self.is_leaf:
            words[self.prefix] = self.index
        for value in self.nodes.values():
            sufixes = value.all_words()
            for suffix, index in sufixes.items():
                words[self.prefix + suffix] = index
        return words

In [2]:
tree = RadixTree()
tree.insert_many(["word", "word1", "word2", "word3", "word12", "abc", "a", "bc"])
print(tree)
tree -= "a"
tree += "c"
tree += "bcd"
tree += "aaa"
tree -= "word12"
tree += "abd"
print(tree.sort(by="index"))
print(tree.sort(by="word"))
print(tree["word"])

{'word': 0, 'word1': 1, 'word12': 4, 'word2': 2, 'word3': 3, 'a': 6, 'abc': 5, 'bc': 7}
[('word', 0), ('word1', 1), ('word2', 2), ('word3', 3), ('abc', 5), ('bc', 7), ('c', 8), ('bcd', 9), ('aaa', 10), ('abd', 11)]
[('aaa', 10), ('abc', 5), ('abd', 11), ('bc', 7), ('bcd', 9), ('c', 8), ('word', 0), ('word1', 1), ('word2', 2), ('word3', 3)]
0


In [3]:
def test_trie() -> bool:
    words = "banana bananas bandana band apple all beast".split()
    root = RadixTree()
    root.insert_many(words)
    words_list = list(root.all_words())
    print(words_list)
    assert len(words) == len(words_list)
    assert words.sort() == words_list.sort()
    assert not root.find("bandanas")[0]
    assert not root.find("apps")[0]
    root.delete("all")
    assert not root.find("all")[0]
    root.delete("banana")
    assert not root.find("banana")[0]
    assert root.find("bananas")[0]
    root += "a"
    assert root.find("a")[0]
    root -= "a"
    assert not root.find("a")[0]
    assert not root.find("apps")[0]
    assert root.find("apple")[0]
    root.print_tree()
    return True


def pytests() -> None:
    assert test_trie()


def main() -> None:
    """
    >>> pytests()
    """
    pytests()


if __name__ == "__main__":
    main()

['banana', 'bananas', 'band', 'bandana', 'beast', 'apple', 'all']
--- b
------ an
--------- anas:    (1)
--------- d:    (3)
------------ ana:    (2)
------ east:    (6)
--- apple:    (4)


In [4]:
class lzw_encoder:
    """
    LZW encoder.
    """

    def __init__(self, by="ascii"):
        """
        Cria um encoder de LZW.

        Args:
            by (str): O tipo de formato. Pode ser 'ascii' ou 'binary'.
        """
        self.next_code = 0
        self.dictionary = RadixTree()
        if by == "binary":
            self.add_to_dictionary("0")
            self.add_to_dictionary("1")
        else:
            for i in range(256):
                self.add_to_dictionary(chr(i))

    def add_to_dictionary(self, word):
        """
        Adiciona uma string ao dicionário.

        Args:
            word (str): A string a ser adicionada.
        """
        self.dictionary += word
        self.next_code += 1

    def encode(self, word, verbose=False):
        """
        Codifica uma string.

        Args:
            word (str): A string a ser codificada.
            verbose (bool): Se True, imprime os valores adicionados.

        Returns:
            list: A lista de códigos.
        """
        r = []
        buffer = ""
        for i in range(len(word)):
            c = word[i]
            if len(buffer) == 0 or (buffer + c) in self.dictionary:
                buffer += c
            else:
                code = self.dictionary[buffer]
                self.add_to_dictionary(buffer + c)
                if verbose:
                    print("added:", buffer + c, "code:", self.next_code - 1)
                buffer = c
                r.append(code)
        if buffer:
            r.append(self.dictionary[buffer])

        return r


class lzw_decoder:
    """
    LZW decoder.
    """

    def __init__(self, by="ascii"):
        """
        Cria um decoder de LZW.

        Args:
            by (str): O tipo de formato. Pode ser 'ascii' ou 'binary'.
        """
        self.next_code = 0
        self.dictionary = {}
        if by == "binary":
            self.add_to_dictionary("0")
            self.add_to_dictionary("1")
        else:
            for i in range(256):
                self.add_to_dictionary(chr(i))

    def add_to_dictionary(self, word):
        """
        Adiciona uma string ao dicionário.

        Args:
            word (str): A string a ser adicionada.
        """
        self.dictionary[self.next_code] = word
        self.next_code += 1

    def decode(self, symbols, verbose=False):
        """
        Decodifica uma lista de códigos.

        Args:
            symbols (list): A lista de códigos.
            verbose (bool): Se True, imprime os valores adicionados.

        Returns:
            str: A string decodificada.
        """
        last_symbol = symbols[0]
        r = self.dictionary[last_symbol]

        for symbol in symbols[1:]:
            if symbol in self.dictionary.keys():
                current = self.dictionary[symbol]
                previous = self.dictionary[last_symbol]
                to_add = current[0]
                self.add_to_dictionary(previous + to_add)
                if verbose:
                    print("added:", previous + to_add, "code:", self.next_code - 1)
                r += current
            else:
                previous = self.dictionary[last_symbol]
                to_add = previous[0]
                self.add_to_dictionary(previous + to_add)
                if verbose:
                    print("added:", previous + to_add, "code:", self.next_code - 1)
                r += previous + to_add
            last_symbol = symbol
        return r

In [5]:
s = "A ASA DA CASA EM CASA"
encoder = lzw_encoder()
encoded = encoder.encode(s)
print("original:", s)
print("encoded:", encoded)

original: A ASA DA CASA EM CASA
encoded: [65, 32, 65, 83, 256, 68, 256, 67, 258, 256, 69, 77, 32, 263, 259]


In [6]:
decoder = lzw_decoder()
decoded = decoder.decode(encoded)
print("decoded:", decoded)

decoded: A ASA DA CASA EM CASA


In [8]:
def str_to_bin(s):
    """
    Converte uma string em ASCII para uma string binária.

    Args:
        s (str): A string a ser convertida.

    Returns:
        bin (str): A string binária.
    """
    bin = ""
    for c in s:
        bin += format(ord(c), "b").zfill(8)
    return bin


def bin_to_str(bin):
    """
    Converte uma string binária para uma string em ASCII.

    Args:
        bin (str): A string binária a ser convertida.

    Returns:
        s (str): A string.
    """
    s = ""
    for i in range(0, len(bin), 8):
        s += chr(int(bin[i : i + 8], 2))
    return s

In [9]:
print(str_to_bin(s))
print(bin_to_str(str_to_bin(s)))

010000010010000001000001010100110100000100100000010001000100000100100000010000110100000101010011010000010010000001000101010011010010000001000011010000010101001101000001
A ASA DA CASA EM CASA


In [10]:
encoder = lzw_encoder(by="binary")
encoded = encoder.encode(str_to_bin(s))
print("original:", s)
print("binary:", str_to_bin(s))
print("encoded:", encoded)

original: A ASA DA CASA EM CASA
binary: 010000010010000001000001010100110100000100100000010001000100000100100000010000110100000101010011010000010010000001000101010011010010000001000011010000010101001101000001
encoded: [0, 1, 0, 4, 4, 3, 2, 5, 5, 7, 10, 8, 7, 1, 3, 11, 6, 18, 9, 8, 19, 21, 12, 23, 24, 6, 16, 20, 28, 14, 28, 9, 14, 20, 23, 30, 27, 21, 17, 22, 2, 32, 12, 32, 15, 25, 2]


In [11]:
decoder = lzw_decoder(by="binary")
decoded = decoder.decode(encoded)
print("binary:", decoded)
print("decoded:", bin_to_str(decoded))

binary: 010000010010000001000001010100110100000100100000010001000100000100100000010000110100000101010011010000010010000001000101010011010010000001000011010000010101001101000001
decoded: A ASA DA CASA EM CASA


In [12]:
def compress(codes_list: list[int], max_bits: int = None) -> tuple[str, int]:
    """
    Comprime uma lista de códigos.

    Args:
        codes_list (list[int]): A lista de códigos.
        max_bits (int): O número máximo de bits.
        Se None, o número máximo de bits é calculado automaticamente.

    Returns:
        bin (str): A string binária compacta.
        max_bits (int): O número máximo de bits.
    """
    if max_bits == None:
        max_bits = max(codes_list).bit_length()
    bin = ""
    for i in codes_list:
        bin += format(i, "b").zfill(max_bits)
    return bin, max_bits


def descompress(bin: str, n_bits: int) -> list[int]:
    """
    Descomprime uma string binária.

    Args:
        bin (str): A string binária a ser descompactada.
        n_bits (int): O número de bits por código.

    Returns:
        codes_list (list[int]): A lista de códigos.
    """
    codes_list = []
    for i in range(0, len(bin), n_bits):
        codes_list.append(int(bin[i : i + n_bits], 2))
    return codes_list

In [13]:
bin_encoded, n_bits = compress(encoded)
bin_encoded, len(bin_encoded)

('000000000001000000000100000100000011000010000101000101000111001010001000000111000001000011001011000110010010001001001000010011010101001100010111011000000110010000010100011100001110011100001001001110010100010111011110011011010101010001010110000010100000001100100000001111011001000010',
 282)

In [14]:
print("Original Size:", len(s) * 8)
print("Compressed Size:", len(bin_encoded))
print("Compression Ratio:", len(s) * 8 / len(bin_encoded))

Original Size: 168
Compressed Size: 282
Compression Ratio: 0.5957446808510638


In [15]:
text = '''class RadixTree:
    """
    Uma árvore de prefixos compacta.
    """

    def __init__(self) -> None:
        """
        Cria uma árvore de prefixos compacta vazia.
        >>> RadixTree()
        """
        # O nó raiz da árvore
        self.root = RadixNode()
        # O número de palavras na árvore
        self.size = 0
        # Próximo índice
        self.next_index = 0

    def insert(self, word: str) -> None:
        """
        Insere uma palavra na árvore.

        Args:
            word (str): palavra a ser inserida.

        >>> RadixTree().insert("word")
        """
        index = self.next_index
        self.root.insert(word, index)
        self.next_index += 1
        self.size += 1

    def insert_many(self, words: list[str]) -> None:
        """
        Insere várias palavras na árvore.

        Args:
            words (list[str]): lista de palavras a serem inseridas.

        >>> RadixTree().insert_many(["word1", "word2"])
        """
        for word in words:
            self.insert(word)

    def find(self, word: str) -> bool:
        """
        Verifica se uma palavra está na árvore.

        Args:
            word (str): palavra a ser verificada.

        Returns:
            bool: True se a palavra estiver na árvore,
                  False caso contrário.

        >>> RadixTree().insert("word")
        >>> RadixTree().find("word")
        True
        """
        return self.root.find(word)

    def index(self, word: str) -> int:
        """
        Retorna o índice de uma palavra na árvore.

        Args:
            word (str): palavra a ser verificada.

        Returns:
            int: índice da palavra na árvore.

        >>> RadixTree().insert("word")
        >>> RadixTree().index("word")
        0
        """
        if self.find(word):
            return self.root.all_words()[word]
        return -1

    def __getitem__(self, word: str) -> int:
        """
        Retorna o índice de uma palavra na árvore.

        Args:
            word (str): palavra a ser verificada.

        Returns:
            int: índice da palavra na árvore.

        >>> RadixTree().insert("word")
        >>> RadixTree()["word"]
        0
        """
        return self.index(word)

    def delete(self, word: str) -> bool:
        """
        Deleta uma palavra da árvore.

        Args:
            word (str): palavra a ser deletada.

        Returns:
            bool: True caso a palavra esteja na árvore e for deletada,
                  False caso a palavra não esteja na árvore.

        >>> RadixTree().insert("word")
        >>> RadixTree().delete("word")
        True
        """
        if self.root.delete(word):
            self.size -= 1
            return True
        return False

    def print_tree(self) -> None:
        """
        Imprime a árvore.

        >>> RadixTree().insert_many(["word1", "word2"])
        >>> RadixTree().print_tree()
        --- word
        ------ 1:   (0)
        ------ 2:   (1)
        """
        self.root.print_tree()

    def all_words(self) -> dict:
        """
        Retorna todas as palavras da árvore numa lista.

        Returns:
            list[(str, int)]: lista de palavras e índices.

        >>> RadixTree().insert_many(["word1", "word2"])
        >>> RadixTree().all_words()
        [('word1', 0), ('word2', 1)]
        """
        return self.root.all_words()

    def __repr__(self) -> str:
        """
        >>> RadixTree()
        """
        return f"RadixTree(size={self.size})"

    def __str__(self) -> str:
        """
        >>> RadixTree()
        """
        return self.all_words().__str__()


    def __contains__(self, word: str) -> bool:
        """
        >>> RadixTree().insert("word")
        >>> "word" in RadixTree()
        True
        """
        return self.find(word)

    def __iter__(self):
        """
        >>> RadixTree().insert_many(["word1", "word2"])
        >>> [(word, i) for (word, i) in RadixTree()]
        [('word1', 0), ('word2', 1)]
        """
        return iter(self.all_words())

    def __add__(self, word: str) -> None:
        """
        >>> RadixTree() + "word"
        >>> "word" in RadixTree()
        True
        """
        self.insert(word)

    def __iadd__(self, word: str):
        """
        >>> RadixTree() += "word"
        >>> "word" in RadixTree()
        True
        """
        self.insert(word)
        return self

    def __sub__(self, word: str) -> bool:
        """
        >>> RadixTree().insert("word")
        >>> RadixTree() - "word"
        >>> "word" not in RadixTree()
        True
        """
        return self.delete(word)

    def __isub__(self, word: str):
        """
        >>> RadixTree().insert("word")
        >>> RadixTree() -= "word"
        >>> "word" not in RadixTree()
        True
        """
        self.delete(word)
        return self

    def __len__(self) -> int:
        """
        >>> RadixTree().insert_many(["word1", "word2"])
        >>> len(RadixTree())
        2
        """
        return self.size

    def sort(self, by='index') -> list[(str, int)]:
        """
        >>> RadixTree().insert_many(["word0", "word1"])
        >>> RadixTree().sort()
        [('word0', 0), ('word1', 1)]
        """
        words = self.all_words()
        words = [(word, i) for word, i in words.items()]
        if by == 'index':
            words.sort(key=lambda x: x[1])
        elif by == 'word':
            words.sort(key=lambda x: x[0])
        return words


class RadixNode:
    """
    Um nó da árvore de prefixos compacta.
    """

    def __init__(self, prefix: str = "", is_leaf: bool = False, index: int = None) -> None:
        """
        Cria um nó da árvore de prefixos compacta.

        Args:
            prefix (str): prefixo do nó.
            is_leaf (bool): se o nó é uma folha.
            index (int): índice da palavra adicionada

        >>> RadixNode()
        >>> RadixNode("prefix")
        >>> RadixNode("prefix", True)
        """
        # Mapeamento a partir do primeiro caractere do prefixo do nó
        self.nodes: dict[str, RadixNode] = {}
        # Um nó será uma folha se a árvore conter sua palavra
        self.is_leaf = is_leaf
        self.prefix = prefix
        self.index = index

    def _match(self, word: str) -> tuple[str, str, str]:
        """
        Compute the common substring of the prefix of the node and a word

        Args:
            word (str): word to compare

        Returns:
            (str, str, str): common substring, remaining prefix, remaining word

        >>> RadixNode("myprefix")._match("mystring")
        ('my', 'prefix', 'string')
        """
        x = 0
        for q, w in zip(self.prefix, word):
            if q != w:
                break

            x += 1

        return self.prefix[:x], self.prefix[x:], word[x:]

    def insert(self, word: str, index: int) -> None:
        # Caso 1: Se a palavra for o prefixo do nó
        # Solução: Definimos o nó atual como folha
        if self.prefix == word and not self.is_leaf:
            self.is_leaf = True

        # Caso 2: O nó não tem arestas que tenham um prefixo para a palavra
        # Solução: Criamos uma aresta do nó atual para um novo
        # contendo a palavra
        elif word[0] not in self.nodes:
            self.nodes[word[0]] = RadixNode(word, True, index)

        else:
            incoming_node = self.nodes[word[0]]
            matching_string, remaining_prefix, remaining_word = incoming_node._match(word)

            # Caso 3: O prefixo do nó é igual ao correspondente
            # Solução: Inserimos a palavra restante no próximo nó
            if remaining_prefix == "":
                self.nodes[matching_string[0]].insert(remaining_word, index)

            # Caso 4: A palavra é maior que a correspondência
            # Solução: Crie um nó entre os dois nós, altere
            # prefixos e adicione o novo nó para a palavra restante
            else:
                incoming_node.prefix = remaining_prefix

                aux_node = self.nodes[matching_string[0]]
                self.nodes[matching_string[0]] = RadixNode(matching_string, False, index)
                self.nodes[matching_string[0]].nodes[remaining_prefix[0]] = aux_node

                if remaining_word == "":
                    self.nodes[matching_string[0]].is_leaf = True
                else:
                    self.nodes[matching_string[0]].insert(remaining_word, index)

    def find(self, word: str) -> bool:
        incoming_node = self.nodes.get(word[0], None)
        if not incoming_node:
            return False
        else:
            matching_string, remaining_prefix, remaining_word = incoming_node._match(word)
            # Se houver prefixo restante, a palavra não pode estar na árvore
            if remaining_prefix != "":
                return False
            # Isso se aplica quando a palavra e o prefixo são iguais
            elif remaining_word == "":
                return incoming_node.is_leaf
            # Temos palavras restantes, então verificamos o próximo nó
            else:
                return incoming_node.find(remaining_word)

    def delete(self, word: str) -> bool:
        incoming_node = self.nodes.get(word[0], None)
        if not incoming_node:
            return False
        else:
            matching_string, remaining_prefix, remaining_word = incoming_node._match(
                word
            )
            # Se houver prefixo restante, a palavra não pode estar na árvore
            if remaining_prefix != "":
                return False
            # Temos palavras restantes, então verificamos o próximo nó
            elif remaining_word != "":
                return incoming_node.delete(remaining_word)
            # Se não for uma folha, não precisamos excluir
            elif not incoming_node.is_leaf:
                return False
            else:
                # Excluímos os nós se nenhuma aresta sair deles
                if len(incoming_node.nodes) == 0:
                    del self.nodes[word[0]]
                    # Nós mesclamos o nó atual com seu único filho
                    if len(self.nodes) == 1 and not self.is_leaf:
                        merging_node = next(iter(self.nodes.values()))
                        self.is_leaf = merging_node.is_leaf
                        self.prefix += merging_node.prefix
                        self.index = merging_node.index
                        self.nodes = merging_node.nodes
                # Se houver mais de uma aresta, nós apenas a marcamos como não-folha
                elif len(incoming_node.nodes) > 1:
                    incoming_node.is_leaf = False
                # Se houver 1 aresta, nós a mesclamos com seu filho
                else:
                    merging_node = next(iter(incoming_node.nodes.values()))
                    incoming_node.is_leaf = merging_node.is_leaf
                    incoming_node.prefix += merging_node.prefix
                    incoming_node.index = merging_node.index
                    incoming_node.nodes = merging_node.nodes
                return True

    def print_tree(self, height: int = 0) -> None:
        if self.prefix != "":
            print("---" * height,(f"{self.prefix}:    ({self.index})" if self.is_leaf else f"{self.prefix}"))

        for value in self.nodes.values():
            value.print_tree(height + 1)

    def all_words(self) -> dict[int, str]:
        words = {}
        if self.is_leaf:
            words[self.prefix] = self.index
        for value in self.nodes.values():
            sufixes = value.all_words()
            for suffix, index in sufixes.items():
                words[self.prefix + suffix] = index
        return words'''

In [16]:
encoder = lzw_encoder(by="ascii")
encoded = encoder.encode(text)
compressed_ascii, n_bits = compress(encoded)

print(f"Original Size:     {len(text)*8} bits")
print(f"Compressed Size:   {len(compressed_ascii)} bits")
print(f"Compression Ratio: {len(text)*8/len(compressed_ascii)}")

Original Size:     93944 bits
Compressed Size:   36336 bits
Compression Ratio: 2.5854249229414354


In [17]:
bin_encoder = lzw_encoder(by="binary")
bin_encoded = bin_encoder.encode(str_to_bin(text))
compressed_bin, n_bits = compress(bin_encoded)

print(f"Original Size:     {len(text)*8} bits")
print(f"Compressed Size:   {len(compressed_bin)} bits")
print(f"Compression Ratio: {len(text)*8/len(compressed_bin)}")

Original Size:     93944 bits
Compressed Size:   87321 bits
Compression Ratio: 1.0758465890221138


In [ ]:
def encoder(text: str, by='binary'):
    encoder = lzw_encoder(by=by)
    t = text
    if by == 'binary':
        t = str_to_bin(t)
    
    encoded = encoder.encode(t)
    compressed, n_bits = compress(encoded)
    print(f"Original Size:     {len(t)*8} bits")
    print(f"Compressed Size:   {len(compressed)} bits")
    print(f"Compression Ratio: {len(t)*8/len(compressed)}")
    return compressed, n_bits

def decoder(code_list, by='binary'):
    decoder = lzw_decoder(by=by)
    